In [26]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.1.0+cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 37.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [27]:
import torch_geometric
from torch_geometric.datasets import Planetoid

#Tutorial1: Introduction

#Load the dataset

In [28]:
dataset = Planetoid(root="tutorial1", name= "Cora")

Processing...
Done!


Dataset properties

In [ ]:
print(dataset)
print(f"Number of graphs:\t\t {len(dataset)}")
print(f"Number of classes:\t\t {dataset.num_classes}")
print(f"Number of node features:\t {dataset.num_node_features}")
print(f"Number of edge features: \t {dataset.num_edge_features}")

Cora()
Number of graphs:		 1
Number of classes:		 7
Number of node features:	 1433
Number of edge features: 	 0


Dataset shapes

In [ ]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


/usr/local/lib/python3.10/dist-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [ ]:
#shape of 2 multiplied by 10556
#node 0 goes into 633, 1862, 2582...
print(f"edge_index:\t\t {dataset.data.edge_index.shape}")
print(dataset.data.edge_index)
print("\n")

#1-d tensor, boolean values that indicate which set is on training,
#validation or test
print(f"train_mask:\t\t {dataset.data.train_mask.shape}")
print(dataset.data.train_mask)
print("\n")

#2708nodes, each node containing 1433 features
print(f"x:\t\t {dataset.data.x.shape}")
print(dataset.data.x)
print("\n")

#node labelling
print(f"y:\t\t {dataset.data.y.shape}")
print(dataset.data.y)

edge_index:		 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])


train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [ ]:
import os.path as osp

import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

In [ ]:
data = dataset[0]

Defining the neural network

In [ ]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()

    self.conv = SAGEConv(dataset.num_features,
                         dataset.num_classes,
                         aggr="max") # specifying the aggregation

  def forward(self):
    x = self.conv(data.x, data.edge_index)
    return F.log_softmax(x, dim=1)

In [ ]:
#to put model on cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
def train():
  model.train()
  optimizer.zero_grad()
  F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
  optimizer.step()

In [ ]:
def test():
  model.eval()
  logits, accs = model(), []
  for _, mask in data("train_mask", "val_mask", "test_mask"):
    pred = logits[mask].max(1)[1]
    acc = pred.eq(data.y[mask]).sum().item()/ mask.sum().item()
    accs.append(acc)
  return accs

In [ ]:
best_val_acc = test_acc = 0
for epoch in range(1, 100):
  train()
  _, val_acc, tmp_test_acc = test()
  if val_acc > best_val_acc:
    best_val_acc = val_acc
    test_acc = tmp_test_acc
  log = "Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}"

  if epoch % 10 == 0:
    print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7280, Test: 0.7190
Epoch: 020, Val: 0.7280, Test: 0.7190
Epoch: 030, Val: 0.7280, Test: 0.7190
Epoch: 040, Val: 0.7280, Test: 0.7190
Epoch: 050, Val: 0.7280, Test: 0.7190
Epoch: 060, Val: 0.7280, Test: 0.7190
Epoch: 070, Val: 0.7280, Test: 0.7190
Epoch: 080, Val: 0.7280, Test: 0.7190
Epoch: 090, Val: 0.7280, Test: 0.7190


#Trying to load own dataset

Embedding the sentences

In [1]:
import pandas as pd

In [ ]:
data = pd.read_csv("agr_en_train.csv")

In [ ]:
data

,POST,Comments,Classification
0,facebook_corpus_msr_1723796,Well said sonu..you have courage to stand agai...,OAG
1,facebook_corpus_msr_466073,"Most of Private Banks ATM's Like HDFC, ICICI e...",NAG
2,facebook_corpus_msr_1493901,"Now question is, Pakistan will adhere to this?",OAG
3,facebook_corpus_msr_405512,Pakistan is comprised of fake muslims who does...,OAG
4,facebook_corpus_msr_1521685,"??we r against cow slaughter,so of course it w...",NAG
...,...,...,...
11994,facebook_corpus_msr_394638,They belong to you flight dirty terrorist coun...,OAG
11995,facebook_corpus_msr_429177,"Really motivating programme, congratulations t...",NAG
11996,facebook_corpus_msr_2032370,fabricated news,OAG
11997,facebook_corpus_msr_1722926,What's wrong with you secular idiots,OAG


In [ ]:
data['Comments']

0        Well said sonu..you have courage to stand agai...
1        Most of Private Banks ATM's Like HDFC, ICICI e...
2           Now question is, Pakistan will adhere to this?
3        Pakistan is comprised of fake muslims who does...
4        ??we r against cow slaughter,so of course it w...
                               ...                        
11994    They belong to you flight dirty terrorist coun...
11995    Really motivating programme, congratulations t...
11996                                      fabricated news
11997                 What's wrong with you secular idiots
11998    Looks like inevitable after all political hard...
Name: Comments, Length: 11999, dtype: object

In [ ]:
!pip install sentence_transformers

In [ ]:
import torch.nn.functional as F

In [ ]:
X = []
for comment in data['Comments'][0:10]:
    X.append(model.encode(comment))

In [ ]:
npX = torch.tensor(X)

In [ ]:
cos = F.cosine_similarity(npX, npX)

In [ ]:
print(cos)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])


In [ ]:
list_of_comments = []

for comment in data["Comments"]:
    list_of_comments.append(comment)

In [ ]:
print(list_of_comments)

['Well said sonu..you have courage to stand against dadagiri of Muslims', "Most of Private Banks ATM's Like HDFC, ICICI etc are out of cash. Only Public sector bank's ATM working", 'Now question is, Pakistan will adhere to this?', 'Pakistan is comprised of fake muslims who does not know the meaning of unity and imposes their thoughts  on others.....all the rascals have gathered there...', '??we r against cow slaughter,so of course it will stop leather manufacturing if it happens.', 'Wondering why Educated Ambassador is struggling to pay through Credit/Debit at a Decent Restaurant! Cant imagine that diplomat of a Developed nation is not having a Card and he needs Cash only for Dinner.', 'How does inflation react to all the after shocks of this demon...?', 'Not good job.....this guis creating a problem n our socacity', 'This is a false news Indian media is simply misguiding there nation and creating hatred..Media should be v careful while spreading the news..SHAME.:(', 'no permanent foes

In [ ]:
print(len(list_of_comments))

11999


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(list_of_comments)

In [ ]:
cos = cosine_similarity(X, X, dense_output = True)
cos = cos - np.eye(cos.shape[0])

In [ ]:
print(cos)

[[ 2.22044605e-16  1.20385853e-01  1.02062073e-01 ...  0.00000000e+00
   1.17851130e-01  0.00000000e+00]
 [ 1.20385853e-01  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  4.25628265e-02]
 [ 1.02062073e-01  0.00000000e+00 -1.11022302e-16 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -2.22044605e-16
   0.00000000e+00  0.00000000e+00]
 [ 1.17851130e-01  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   2.22044605e-16  1.66666667e-01]
 [ 0.00000000e+00  4.25628265e-02  0.00000000e+00 ...  0.00000000e+00
   1.66666667e-01 -1.11022302e-16]]


In [ ]:
print(cos[0])

[2.22044605e-16 1.20385853e-01 1.02062073e-01 ... 0.00000000e+00
 1.17851130e-01 0.00000000e+00]


In [ ]:
print(len(cos[0]))

11999


In [ ]:
for lst in cos:
    for num in lst:
        if num < -0.5:
            print(num)

-1.0
-1.0
-1.0
-1.0
-1.0


#The dataset to work with

In [7]:
cyberbully_df = pd.read_csv("cyberbully.csv")

In [8]:
cyberbully_df

,Img_Name,Img_Text,Img_Text_Label,Img_Label,Text_Label,Sentiment,Emotion,Sarcasm,Harmful_Score,Target,Unnamed: 10,Unnamed: 11
0,0.jpg,Shivam @shivamishraa Girls be named naina and ...,Bully,Nonbully,Bully,Negative,Disgust,Yes,Partially-Harmful,Individual,NaN,NaN
1,1.jpg,Aaloo ke paranthe is the best breakfast Omelet...,Nonbully,Nonbully,Nonbully,Neutral,Other,No,Harmless,None,NaN,NaN
2,2.jpg,For Boyfriend For Bestfriend DESI ADUKT TROLLS,Bully,Bully,Nonbully,Negative,Ridicule,No,Partially-Harmful,Society,NaN,NaN
3,3.jpg,You find a new YouTuber He's funny All of his ...,Nonbully,Nonbully,Nonbully,Neutral,Sadness,No,Harmless,None,NaN,NaN
4,4.jpg,not_shubham14 @mentally_dank Kids at Marine Dr...,Bully,Nonbully,Nonbully,Negative,Sadness,No,Partially-Harmful,Individual,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5860,6001.jpg,Corona Vaccine Ki Testing Bezubaan janwar Par ...,Bully,Nonbully,Bully,Negative,Angry,No,Very-Harmful,Society,NaN,NaN
5861,6002.jpg,""" WHY DO YOU REPLY EVERYONE INSTANTLY?"" BECAU...",Nonbully,Nonbully,Nonbully,Neutral,Sadness,No,Harmless,None,NaN,NaN
5862,6003.jpg,Girls after reading desi adult trolls memes,Bully,Nonbully,Bully,Negative,Ridicule,Yes,Partially-Harmful,Community,NaN,NaN
5863,6004.jpg,Jumping of this would solve all of my problems,Nonbully,Nonbully,Nonbully,Positive,Anticipation,No,Harmless,None,NaN,NaN


In [9]:
cyberbully_df["Img_Text"]

0       Shivam @shivamishraa Girls be named naina and ...
1       Aaloo ke paranthe is the best breakfast Omelet...
2          For Boyfriend For Bestfriend DESI ADUKT TROLLS
3       You find a new YouTuber He's funny All of his ...
4       not_shubham14 @mentally_dank Kids at Marine Dr...
                              ...                        
5860    Corona Vaccine Ki Testing Bezubaan janwar Par ...
5861    " WHY DO YOU REPLY  EVERYONE INSTANTLY?" BECAU...
5862          Girls after reading desi adult trolls memes
5863       Jumping of this would solve all of my problems
5864        Farrago Abdullah @abdullah_Omar Who did this 
Name: Img_Text, Length: 5865, dtype: object

In [20]:
cyberbully_arr = np.array(cyberbully_df)

In [18]:
cyberbully_df = cyberbully_df["Img_Text"].dropna()

In [21]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(cyberbully_arr)

In [22]:
cos = cosine_similarity(X, X, dense_output = True)
cos = cos - np.eye(cos.shape[0])

In [23]:
cos

array([[-2.22044605e-16,  0.00000000e+00,  0.00000000e+00, ...,
         1.04828484e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.33066907e-16,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.51976315e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.04828484e-01,  0.00000000e+00,  2.51976315e-01, ...,
        -3.33066907e-16,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  1.23091491e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.23091491e-01,  2.22044605e-16]])

In [25]:
for lst in cos:
    for num in lst:
        if num > 0.9:
            print(num)

0.9555330859059092
0.9999999999999999
0.9354143466934851
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999999
0.9333333333333331
1.0
1.0000000000000004
0.9999999999999998
1.0000000000000002
0.9999999999999997
0.9999999999999999
0.9999999999999999
1.0
1.0000000000000002
0.9999999999999998
0.9999999999999997
1.0000000000000002
1.0
0.9655172413793102
0.9655172413793102
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
1.0000000000000002
0.9999999999999999
1.0
0.9999999999999999
0.9999999999999997
0.9999999999999997
1.0
0.9583333333333336
0.9583333333333336
0.9583333333333336
0.9999999999999998
0.9999999999999997
0.9999999999999996
0.9473684210526312
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9999999999999999
0.9583333333333336
1.0000000000000002
1.0000000000000002
1.0
0.9128709291752769
0.9999999999999999
0.9198662110078001
0.9090909090909093
1.0
0.9999999999999999
0.9999999999999999
1.0
0.999999999999